# Application State Machine
We're going to build a state machine to handle applications. The basic flow and constrains of a application is as below:
1. Application has following attributes:
  - `General information`, such as application's owner info, terms, etc.
  - `Signatures`, a set of signatures of owners', witness, etc.
  - `Expiry Date`, when application to be expired
  - `Payment info`, info relating to payment process
  - `Underwriting info`, info relating to underwriting process
2. Possible actions:
  - `update`, update `general information` of the application
  - `sign`, sign the application, update `Signatures`
  - `underwriting`, send the application through underwriting process
  - `pay`, send the application through payment process
  - `submit`, submit the final application
3. Possible states of application and respective constrains:

| States | Contrains |
|:----| :-|
|Draft| Can only update `general information` or provide `signatures`|
|Signed| Cannot update `General information`, can sign to complete `signatures`<br>can do underwriting|
|Underwriting| pending for underwriting result<br> can only `underwriting`|
|Underwritten| can only proceed to payment|
|Payment_Pending| can only update payment info|
|Payment_Done| payment successfully, can only submit |
|Payment_Failed| Last payment was failed, can pay again|
|Submitted| Application was submitted, accepting state|

|State          | update<br>general info| sign       |underwriting(req)| underwriting(res)|pay(req)| pay(res_ok) | Submit |
|:---           |:---                   |:---        |:---             |:---        |:---         |:---            |:---|
|Draft          |Draft                  | Signed     |                 |            |             |                |         |
|Signed         |N/A                    |Signed      |Underwriting     |            |             |                |         |
|Underwriting   |N/A                    |N/A         |Underwriting     |Underwritten|             |                |         |
|Underwritten   |N/A                    |            |                 |Underwritten|Payment_pending|             |         |
|Payment_pending|N/A                    |            |Underwriting     |            |             |Payment_Done|         |
|Payment_Done   |N/A                    |            |Underwriting     |            |             |              |Submitted|
|Payment_Failed |N/A                    |            |Underwriting     |            |Payment_pending|Payment_Done|         |
|Submitted      |N/A                    |            |Underwriting     |            |             |                |         |

# Class design
## ApplicationModel
Represent an application with all required data
## ApplicationStateMachine
The state machine that will handle event and input to update application accordingly
## State
Represent application's state. Each state class will have `enter` and `exit` actions. `enter` is triggered when application enters the state, and `exit` is triggered when application exits the state.
## Predicate
Check the validity of the input
## Input
Represent different kind of input

In [118]:
class ApplicationModel:
    
    def __init__(self, name=None, expiry_date=None, roles=None, signatures=None, 
                 underwriting_info=None, payment_info=None, submission_info=None):
        self._name = name
        self._expiry_date = expiry_date
        if not roles:
            roles = []
        self._roles = roles
        if not signatures:
            signatures = {}
        self._signatures = signatures
        self._underwriting_info = underwriting_info
        self._payment_info = payment_info
        self._submission_info = submission_info
        self._state = 'Draft'
        
    def __str__(self):
        return (f'''
        Application info
        Name: {self._name}
        Signatures: {self._signatures}
        State: {self._state!r}
        Roles: {self._roles}
        ''')

In [156]:
import abc


class State(abc.ABC):
    _state_name = "State"
    
    @property
    def state_name(kls):
        return kls._state_name
    
    def enter(self, application: ApplicationModel, _input:ApplicationModel):
        '''Entering state'''
        print(f'Entering {self.state_name}')
        application._state = self.state_name
        self._enter_action(application, _input)
        
        return
        
    def exit(self, application: ApplicationModel, _input:ApplicationModel):
        '''existing state'''
        print(f'Existing {self.state_name}')
        self._exit_action(application, _input)
        
        return
    
    @abc.abstractmethod
    def _enter_action(self, application: ApplicationModel, _input: ApplicationModel):
        return None
    
    @abc.abstractmethod
    def _exit_action(self, application: ApplicationModel, _input: ApplicationModel):
        return None
    
    @classmethod
    def get_state_name(kls):
        return kls._state_name
    
    def __init_subclass__(subclass, *args, **kwargs):
        print('__init_subclass__', subclass, args, kwargs)
        if "_state_name" not in subclass.__dict__:
            raise Exception("Need to implement attribute _state_name")
        return super().__init_subclass__(*args, **kwargs)
    
    def __str__(self):
        return self._state_name
        
    def __repr__(self):
        return self._state_name
        
class DraftState(State):
    '''
    Represent Draft state
    While the application is draft, user could update general info.
    '''
    _state_name = "Draft"
    
    def _enter_action(self, application: ApplicationModel, _input: ApplicationModel):
        '''Update application data according to the input application'''
        application._name = _input._name
        
        return None
        
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        '''Exiting Draft, check for data completeness before proceed to other state'''
        if not application._name:
            raise Exception("Missing application name")
        
        return None

class SignedState(State):
    '''
    The application is signed upon received the first signature.
    After the application is signed, user cannot update general info
    Once all necessary signatures are acquired, further attempt to sign will fail
    Attemp to override signature will also fail. This should be checked before entering this state.
    '''
    _state_name = "Signed"
        
    def _enter_action(self, application: ApplicationModel, _input: ApplicationModel):
        '''Update application data'''
        application._signatures.update(_input._signatures)
        
        return None
        
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        '''Exiting Draft'''
        return None

__init_subclass__ <class '__main__.DraftState'> () {}
__init_subclass__ <class '__main__.SignedState'> () {}


In [157]:
class UnderwritingState(State):
    _state_name = "Underwriting"
    
    def _enter_action(self, application: ApplicationModel, _input: ApplicationModel):
        '''Underwriting application'''
        
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        '''Exiting Underwriting'''
        return None
    
    
class UnderwrittenState(State):
    _state_name = "Underwritten"
    
    def _enter_action(self, application: ApplicationModel, _input: ApplicationModel):
        '''Underwriting application'''
        
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        '''Exiting Underwriting'''
        return None

__init_subclass__ <class '__main__.UnderwritingState'> () {}
__init_subclass__ <class '__main__.UnderwrittenState'> () {}


In [187]:
class PaymentPendingState(State):
    _state_name = "Payment_Pending"
    
    def _enter_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass
    
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass
    
    
class PaymentSuccessState(State):
    _state_name = "Payment_Success"
    
    def _enter_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass
    
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass
    

class PaymentFailedState(State):
    _state_name = "Payment_Failed"
    
    def _enter_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass
    
    def _exit_action(self, application: ApplicationModel, _input:ApplicationModel):
        pass

__init_subclass__ <class '__main__.PaymentPendingState'> () {}
__init_subclass__ <class '__main__.PaymentSuccessState'> () {}
__init_subclass__ <class '__main__.PaymentFailedState'> () {}


In [225]:
class SubmittedState(State):
    _state_name = "Submitted"
    
    def _enter_action(self, *args, **kwargs):
        pass
    
    def _exit_action(self, *args, **kwargs):
        pass

__init_subclass__ <class '__main__.SubmittedState'> () {}


In [226]:
class StateStore:
    '''
    StateStore act as a State repository, provide a initialized state object
    State objects could be shared accross system.
    The key to identify the state object is the state name.
    '''
    draft_state = DraftState()
    signed_state = SignedState()
    underwriting_state = UnderwritingState()
    underwritten_state = UnderwrittenState()
    payment_pending_state = PaymentPendingState()
    payment_failed_state = PaymentFailedState()
    payment_success_state = PaymentSuccessState()
    SubmittedState = SubmittedState()
    
    states = {
        draft_state.state_name: draft_state,
        signed_state.state_name: signed_state,
        underwriting_state.state_name: underwriting_state,
        underwritten_state.state_name: underwritten_state,
        payment_pending_state.state_name: payment_pending_state,
        payment_failed_state.state_name: payment_failed_state,
        payment_success_state.state_name: payment_success_state,
        SubmittedState.state_name: SubmittedState,
    }
    @classmethod
    def get_state(kls, state_name):
        # Simple handling for undefined state.
        if state_name not in kls.states:
            raise Exception(f"{state_name} isn't supported")
        return kls.states[state_name]
    

## Decision making
Business rules regarding signatures:
1. After the first signature is acquired, no further update to application's general info is allowed.
2. Not allow to overwrite signatures
3. Only after all signatures are acquired, can proceed to Underwriting

For rule #1, we can simply config the `state_table` so that `update` event are not allowed when application's state is `Signed`.

As for rule #2, to check whether a signature was captured before, we need access to the application. There are two ways:
- Include the check in `enter` method of `Signed` state, since we have both input application and the application itself. The checking logic is also closely relating to capturing the signature. However, if there're many validations like this, we will very populate this state class. It will be difficult to add or remove validation logic if required.
- Put the checking logic to a separate `Predicate` subclass. This approach will alow us to config the validation rules as needed. Also keeping the state subclass clear. However, will lead to class explosions if there are many validation rule.

We will follow the second approach in this case.

In [138]:
class Predicate(abc.ABC):
    
    @abc.abstractmethod
    def check(self):
        return False
    

class NoOverwrittenSingaturePredicate(Predicate):
    
    def check(self, application: ApplicationModel, _input: ApplicationModel):
        current_signature = set(application._signatures)
        new_signature = set(_input._signatures)
        overwritten_signature = current_signature & new_signature
        if overwritten_signature:
            raise Exception(f"Signature of {overwritten_signature} was captured already, cannot overwrite")
        
        return True

Rule #3 seems to be tricky at first. 

However, in our sample machine, since we don't have a dedicated state to indicate the application is fully signed or partially signed, there's will be no state transition. There's also no other actions required when we captured all signatures. We already have `NoOverwrittenSignaturePredicate` in place, so there no worry about client keep calling `sign` again.

According to rule #3, we only need to ensure that all signatures are captured before event `underwriting` (which will be invoked when entering `underwriting` state, so we could make another `predicate` to check this before entering state `underwriting`.

## Input type
So far, we're using only one type of input: `ApplicationModel`, assuming that this input will provide all necessary infomation to for all event. This seems to be, at first, convenient for the state machine however, has some drawbacks:
1. The `ApplicationModel` class might be used in a awkward way. To prepare the input for each event, the client/controller (the code that invoke our state machine) need to initialize a partial `ApplicationModel`:
  - to `update`, an `ApplicationModel` is initialized but armed with only `Generate Information`
  - to `sign`, an `ApplicationModel` is initialized but armed with only the `signatures` of the person who is signing
  - to `underwriting`, there should be no input required, but we need to be careful about how to handle this scenario.
    - **Removing the `input` is not a good idea** since it's not consistent with other combination and further more, we will need branching in our state machine to check whether we need to invoke the machine with or without `input`.
    - Another possible solution is to **pass an empty `ApplicationModel`**, this works and is awkward.
    - The most suitable solution is to **create a new `input` class**, maybe just a dummy one, just for the sake of distinguish different kind of `input`. We also need to create another `input` class to represent the `underwriting` result. 
    
We have more class but that also makes our code more expressive. By looking at `state_table`, we know what `input` is expected.

### Payment input
For payment, we have 4 scenarios:
1. Application is in underwritten state, start the payment, state is `payment_pending`
2. Payment is in process, start another payment, should failed
3. Payment failed, state is `payment_failed`
4. Payment successful, state is `payment_successful`

Scenario #1 is a simple state transition.

Scenario #2 is also a simple state transition in which we prohibit they action `pay` when the state is `payment_pending`. Or we could allow multiple `payment session`, as long as the **session** share a common identifier so that the payment engine downstream could prevent **double payment**. The payment process itself is a different story, probably a long one but it does influent how we handle the state. To make life easier, we will prohibit multiple payment session.

Scenario #3 and #4 has a small catch: how we're gonna config the `state_table`?<br>
- Branching the flow based on `event` or `input` will be straight forward with some caveats: 
  - in both case, the controller will decide which action to invoke in the State Machine. This is less intuitive, and undermine the state machine's power. 
>Imagine that you have to select which elavators for your ride after entering floor destination. Not very convenient, right?
>Or a vending machine that has different slots for different notes.
- How about using `Predicate` to control the flow? We could check the input and allow it to match an appropriate `target_state`. Let's enhance our state machine.

# TODO
- We have only one generic error message when a particular combination of (`current_state`, `event`, `input`) are not supported. If there's a more specific message help us to determine if `event` or `input` is invalid, that would be better.
- How would our machine perform in a concurrency environment?


In [228]:
class UnderwritingReqInput:
    pass


class UnderwritingResInput:
    pass

class PaymentInput:
    pass

class PaymentFailedInput:
    pass
        
class PaymentSuccessInput:
    pass

class SubmitInput:
    pass

In [194]:
class AllSignaturesCapturedPredicate(Predicate):
    
    def check(self, application: ApplicationModel, _input: ApplicationModel):
        current_signatures = set(application._signatures)
        required_signatures = set(application._roles)
        
        if not required_signatures == current_signatures:
            raise Exception(f"Missing signature of {required_signatures - current_signatures}")
            
        return True

In [234]:
class ApplicationMachine:
    
    state_table = {}
    
    def __init__(self, application: ApplicationModel):
        self._application = application
        self._state = StateStore.get_state(application._state)
        
    def handle(self, event, _input):
        current_state = self._state
        key = (self._state.__class__, event, _input.__class__)
        # Raise error if the state, event, input are not defined.
        # So that we don't need to care about the machine messes up the state.
        if key not in self.state_table:
            print(key)
            err_msg = f"Cannot {event} when application is {current_state!r}"
            raise Exception(err_msg)
        
        # Get the set of predicates, and target_state for corresponding input
        predicates, target_state = self.state_table.get(key, (None, None))
        # Run all the checks
        for predicate in predicates:
            if not predicate.check(self._application, _input):
                print(f"{predicate} failed")
                return
        
        # Check whether we're transitioning to different state, if so, calling exit of current state
        if target_state and target_state != self._state:
            self._state.exit(self._application, _input)
            self._state = StateStore.get_state(target_state.get_state_name())
            
        print(self._state)
        # Execute the action when entering some state. Reentering the current state also invoke this action.
        self._state.enter(self._application, _input)
        
        return
    
    def print_app(self):
        '''A simple helper to check application status'''
        print(self._application)
        
ApplicationMachine.state_table={
    (DraftState, 'update', ApplicationModel): ([], None),
    (DraftState, 'sign', ApplicationModel): ([], SignedState),
    (SignedState, 'sign', ApplicationModel): ([NoOverwrittenSingaturePredicate()], SignedState),
    (SignedState, 'underwriting', UnderwritingReqInput): ([AllSignaturesCapturedPredicate()], UnderwritingState),
    (UnderwritingState, 'underwriting', UnderwritingResInput): ([], UnderwrittenState),
    (UnderwrittenState, 'pay', PaymentInput): ([], PaymentPendingState),
    (PaymentPendingState, 'pay', PaymentFailedInput): ([], PaymentFailedState),
    (PaymentPendingState, 'pay', PaymentSuccessInput): ([], PaymentSuccessState),
    (PaymentFailedState, 'pay', PaymentInput): ([], PaymentPendingState),
    (PaymentSuccessState, 'submit', SubmitInput): ([], SubmittedState),
}

In [204]:
class PaymentResultInput:
    
    def __init__(self, payment_success):
        self.payment_success = payment_success

In [205]:
class IsPaymentFailedPredicate(Predicate):
    
    def check(self, application: ApplicationModel, _input: PaymentResultInput):
        return not _input.payment_success
    
    
class IsPaymentSuccessPredicate(Predicate):
    
    def check(self, application: ApplicationModel, _input: PaymentResultInput):
        return _input.payment_success

In [235]:
import unittest

app = ApplicationModel(roles=['An', 'Vien'])
app_machine = ApplicationMachine(app)
app_machine.print_app()
events_inputs = [
    ('update', ApplicationModel(name="An"), "Draft"),
    ('update', ApplicationModel(name="Vien"), "Draft"),
    ('sign', ApplicationModel(signatures={"An": "An's signature"}), "Signed"),
    ('update', ApplicationModel(name="An"), Exception),
    ('sign', ApplicationModel(signatures={"An": "An's second signature"}), Exception),
    ('underwriting', UnderwritingReqInput(), Exception),
    ('sign', ApplicationModel(signatures={"Vien": "Vien's signature"}), "Signed"),
    ('underwriting', UnderwritingReqInput(), "Underwriting"),
    ('underwriting', UnderwritingResInput(), "Underwritten"),
    ('pay', PaymentInput(), "Payment_Pending"),
    ('pay', PaymentFailedInput(), "Payment_Failed"),
    ('pay', PaymentInput(), "Payment_Pending"),
    ('pay', PaymentSuccessInput(), "Payment_Success"),
    ('submit', SubmitInput(), "Submitted")
]

print(ApplicationMachine.state_table)
for event, _input, expect in events_inputs:
    print(f"Attempt to {event}")
    if expect is Exception:
        with unittest.TestCase.assertRaises('', Exception) as e:
            app_machine.handle(event, _input)
        print(e.exception)
    else:
        app_machine.handle(event, _input)
        assert app_machine._application._state == expect, "Something wrong"
    
    app_machine.print_app()
    



        Application info
        Name: None
        Signatures: {}
        State: 'Draft'
        Roles: ['An', 'Vien']
        
{(<class '__main__.DraftState'>, 'update', <class '__main__.ApplicationModel'>): ([], None), (<class '__main__.DraftState'>, 'sign', <class '__main__.ApplicationModel'>): ([], <class '__main__.SignedState'>), (<class '__main__.SignedState'>, 'sign', <class '__main__.ApplicationModel'>): ([<__main__.NoOverwrittenSingaturePredicate object at 0x000001E13E15FE20>], <class '__main__.SignedState'>), (<class '__main__.SignedState'>, 'underwriting', <class '__main__.UnderwritingReqInput'>): ([<__main__.AllSignaturesCapturedPredicate object at 0x000001E13E15F7F0>], <class '__main__.UnderwritingState'>), (<class '__main__.UnderwritingState'>, 'underwriting', <class '__main__.UnderwritingResInput'>): ([], <class '__main__.UnderwrittenState'>), (<class '__main__.UnderwrittenState'>, 'pay', <class '__main__.PaymentInput'>): ([], <class '__main__.PaymentPendingState'>), (

In [232]:
class ApplicationMachineV2:
    
    state_table = {}
    
    def __init__(self, application: ApplicationModel):
        self._application = application
        self._state = StateStore.get_state(application._state)
        
    def handle(self, event, _input):
        current_state = self._state
        key = (self._state.__class__, event, _input.__class__)
        # Raise error if the state, event, input are not defined.
        # So that we don't need to care about the machine messes up the state.
        if key not in self.state_table:
            print(key)
            err_msg = f"Cannot {event} when application is {current_state!r}"
            raise Exception(err_msg)
        # In order to ensure backward compatible, we need to treat the value of a particular key
        # in the state_table a single possible transition, not a list of transitions
        transitions = self.state_table.get(key, (None, None))
        if not isinstance(transitions, list):
            transitions = [transitions]
        
        for transition in transitions:
            # Get the set of predicates, and target_state for corresponding input
            predicates, target_state = transition
            # Run all the checks
            check = True
            for predicate in predicates:
                if not predicate.check(self._application, _input):
                    print(f"{predicate} failed")
                    check = False
                    break
            
            if not check:
                continue

            # Check whether we're transitioning to different state, if so, calling exit of current state
            if target_state and target_state != self._state:
                self._state.exit(self._application, _input)
                self._state = StateStore.get_state(target_state.get_state_name())

            print(self._state)
            # Execute the action when entering some state. Reentering the current state also invoke this action.
            self._state.enter(self._application, _input)
            
            return
        
        return
    
    def print_app(self):
        '''A simple helper to check application status'''
        print(self._application)
        
ApplicationMachineV2.state_table={
    (DraftState, 'update', ApplicationModel): ([], None),
    (DraftState, 'sign', ApplicationModel): ([], SignedState),
    (SignedState, 'sign', ApplicationModel): ([NoOverwrittenSingaturePredicate()], SignedState),
    (SignedState, 'underwriting', UnderwritingReqInput): ([AllSignaturesCapturedPredicate()], UnderwritingState),
    (UnderwritingState, 'underwriting', UnderwritingResInput): ([], UnderwrittenState),
    (UnderwrittenState, 'pay', PaymentInput): ([], PaymentPendingState),
    (PaymentPendingState, 'pay', PaymentResultInput): [
        ([IsPaymentFailedPredicate()], PaymentFailedState),
        ([IsPaymentSuccessPredicate()], PaymentSuccessState)
    ],
    (PaymentFailedState, 'pay', PaymentInput): ([], PaymentPendingState),
    (PaymentSuccessState, 'submit', SubmitInput): ([], SubmittedState),
}

In [233]:
# Test ApplicationMachineV2
import unittest

app = ApplicationModel(roles=['An', 'Vien'])
app_machine = ApplicationMachineV2(app)
app_machine.print_app()
events_inputs = [
    ('update', ApplicationModel(name="An"), "Draft"),
    ('update', ApplicationModel(name="Vien"), "Draft"),
    ('sign', ApplicationModel(signatures={"An": "An's signature"}), "Signed"),
    ('update', ApplicationModel(name="An"), Exception),
    ('sign', ApplicationModel(signatures={"An": "An's second signature"}), Exception),
    ('underwriting', UnderwritingReqInput(), Exception),
    ('sign', ApplicationModel(signatures={"Vien": "Vien's signature"}), "Signed"),
    ('underwriting', UnderwritingReqInput(), "Underwriting"),
    ('underwriting', UnderwritingResInput(), "Underwritten"),
    ('pay', PaymentInput(), "Payment_Pending"),
    ('pay', PaymentResultInput(payment_success=False), "Payment_Failed"),
    ('pay', PaymentInput(), "Payment_Pending"),
    ('pay', PaymentResultInput(payment_success=True), "Payment_Success"),
    ('submit', SubmitInput(), "Submitted")
]

print(ApplicationMachine.state_table)
for event, _input, expect in events_inputs:
    print(f"Attempt to {event}")
    if expect is Exception:
        with unittest.TestCase.assertRaises('', Exception) as e:
            app_machine.handle(event, _input)
        print(e.exception)
    else:
        app_machine.handle(event, _input)
        current_state = app_machine._application._state
        assert current_state == expect, f"Expect {expect} got {current_state}"
    
    app_machine.print_app()


        Application info
        Name: None
        Signatures: {}
        State: 'Draft'
        Roles: ['An', 'Vien']
        
{(<class '__main__.DraftState'>, 'update', <class '__main__.ApplicationModel'>): ([], None), (<class '__main__.DraftState'>, 'sign', <class '__main__.ApplicationModel'>): ([], <class '__main__.SignedState'>), (<class '__main__.SignedState'>, 'sign', <class '__main__.ApplicationModel'>): ([<__main__.NoOverwrittenSingaturePredicate object at 0x000001E13CBABCA0>], <class '__main__.SignedState'>), (<class '__main__.SignedState'>, 'underwriting', <class '__main__.UnderwritingReqInput'>): ([<__main__.AllSignaturesCapturedPredicate object at 0x000001E13CBAB8E0>], <class '__main__.UnderwritingState'>), (<class '__main__.UnderwritingState'>, 'underwriting', <class '__main__.UnderwritingResInput'>): ([], <class '__main__.UnderwrittenState'>), (<class '__main__.UnderwrittenState'>, 'pay', <class '__main__.PaymentInput'>): ([], <class '__main__.PaymentPendingState'>), (

# Summary
We have finished implement a simplified version of a state machine to control Application's state. Regardless of its simplicity, we have covered most of potential scenarios and identified some paint points (where multiple options are available) in this machine. This allow us to further enhance the machine as more use cases to be requested. Below are primary components:
1. The `ApplicationMachine` is our main working horse, centralized control that piercing all the piece together.
2. The `State` subclasses, representing all possible states of an application. So far, we got: `Draft`, `Signed`, `Underwriting`, `Underwritten`, `Payment_Pending`, `Payment_Failed`, `Payment_Success`, `Submitted`.<br>
   Each of these states will handle its own operation via `_enter_action` and `_exit_action`. None of them are aware of the other states, making them **independent and decoupling** from each other.
3. The `Input` classes define different type of inputs, making the `state_table` more expressive and specific.
4. The `Predicates` classes define conditional logic to offer validations, flow controls.

# Further enhancement
1. The only component we haven't standardized is `event`. We're using string to identify different event. This could be organized with an `Enum` class.
2. The `state_table` is quite primitive. We could also standadized the table format with `namedtuple` or a dedicated class.
3. Generalizing the `StateMachine` so it could be used for different kind of object. As long as we define all related components, i.e. `State`, `Input`, `Predicates`, and respective `state_table`.
4. The handling of missing State, Predicate reject/acceptance would also be further enhanced
5. One major area we definitely need to look into is how consistent our machine is, i.e. what is the posibility it would leave the obj in invalid state?